In [90]:
import inspect
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, ConversationChain
from langchain.chains.conversation.memory import (ConversationBufferMemory,
                                                  ConversationSummaryMemory,
                                                  ConversationBufferWindowMemory,
                                                  ConversationKGMemory)
from langchain.callbacks import get_openai_callback
import tiktoken

In [91]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [92]:
llm = OpenAI(
    temperature=0,
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-3.5-turbo'  # can be used with llms like 'gpt-3.5-turbo'
)

In [93]:
def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

In [94]:
conversation = ConversationChain(
    llm=llm,
)

Option 1:  Simple raw memory of conversation

In [95]:
conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

In [96]:
conversation_buf.memory.save_context({"input": "Hello"}, {"output": "How are you?"})
conversation_buf.memory.save_context({"input": "Doing well, and you?"}, {"output": "I'm good too, thank you for asking!"})

conversation_buf("Good morning GPT-3.5-turbo!")

{'input': 'Good morning GPT-3.5-turbo!',
 'history': "Human: Hello\nAI: How are you?\nHuman: Doing well, and you?\nAI: I'm good too, thank you for asking!",
 'response': 'Good morning to you too! How can I assist you today?'}

In [97]:
count_tokens(
    conversation_buf,
    "I'm interested in understanding the structure of an electron, and to find out some details about subatomic particles"
)

Spent a total of 232 tokens


'Sure, I can help with that! Electrons are subatomic particles that have a negative charge and are found in the electron cloud surrounding the nucleus of an atom. They are considered to be elementary particles, meaning they cannot be broken down into smaller components. Other subatomic particles include protons, which have a positive charge, and neutrons, which have no charge. Is there anything specific you would like to know about these particles?'

In [98]:
count_tokens(
    conversation_buf,
    "I'd like to understand the approximate size of these particles and how they relate to other subatomic particles like quarks and the Higgs-Boson"
)

Spent a total of 420 tokens


'Sure, I can provide some information on that. Electrons are incredibly small, with a radius of approximately 2.8 x 10^-15 meters. Protons and neutrons are slightly larger, with radii of approximately 1 x 10^-15 meters. Quarks, which are the building blocks of protons and neutrons, are even smaller, with radii of approximately 10^-18 meters. The Higgs-Boson, on the other hand, is a subatomic particle that was discovered in 2012 and is responsible for giving other particles mass. It has a mass of approximately 125 giga-electronvolts (GeV) and a radius of approximately 10^-18 meters. Does that help answer your question?'

In [99]:
count_tokens(
    conversation_buf,
    "Yes it does.   Are there smaller particles than the ones you have already mentioned?"
)

Spent a total of 496 tokens


'As far as we currently know, electrons, protons, neutrons, and quarks are considered to be the smallest subatomic particles. However, there is ongoing research in the field of particle physics to discover even smaller particles or to better understand the properties of these particles.'

In [100]:
count_tokens(
    conversation_buf,
    "remind me what I am seeking help with?"
)

Spent a total of 547 tokens


'You were interested in understanding the structure of an electron and learning about subatomic particles, including their size and how they relate to each other. Is there anything else I can help you with?'

In [101]:
print(conversation_buf.memory.buffer)

Human: Hello
AI: How are you?
Human: Doing well, and you?
AI: I'm good too, thank you for asking!
Human: Good morning GPT-3.5-turbo!
AI: Good morning to you too! How can I assist you today?
Human: I'm interested in understanding the structure of an electron, and to find out some details about subatomic particles
AI: Sure, I can help with that! Electrons are subatomic particles that have a negative charge and are found in the electron cloud surrounding the nucleus of an atom. They are considered to be elementary particles, meaning they cannot be broken down into smaller components. Other subatomic particles include protons, which have a positive charge, and neutrons, which have no charge. Is there anything specific you would like to know about these particles?
Human: I'd like to understand the approximate size of these particles and how they relate to other subatomic particles like quarks and the Higgs-Boson
AI: Sure, I can provide some information on that. Electrons are incredibly smal

Option 2: Memory is a summary of prior conversation to be economical on tokens


In [102]:
conversation_sum = ConversationChain(
    llm=llm,
    memory=ConversationSummaryMemory(llm=llm)
)

In [103]:
conversation_sum.memory.save_context({"input": "Hello"}, {"output": "How are you?"})
conversation_sum.memory.save_context({"input": "Doing well, and you?"}, {"output": "I'm good too, thank you for asking!"})

In [104]:
count_tokens(
    conversation_sum,
    "Good morning GPT-3.5-turbo!"
)

Spent a total of 365 tokens


'Good morning to you too! How are you doing today?'

In [105]:
count_tokens(
    conversation_sum,
    "I'm interested in understanding the structure of an electron, and to find out some details about subatomic particles"
)

Spent a total of 632 tokens


"Ah, that's a fascinating topic! Electrons are subatomic particles that have a negative charge and are found in the electron cloud surrounding the nucleus of an atom. They are considered to be point particles, meaning they have no size or structure. However, they do have properties such as spin and magnetic moment that can be measured. As for other subatomic particles, there are many different types such as protons, neutrons, and various types of mesons and baryons. Is there anything specific you would like to know?"

In [106]:
count_tokens(
    conversation_sum,
    "I'd like to understand the approximate size of these particles and how they relate to other subatomic particles like quarks and the Higgs-Boson"
)

Spent a total of 919 tokens


"Sure, I can help with that. Electrons are considered point particles, meaning they have no size or structure. Protons and neutrons, on the other hand, are made up of smaller particles called quarks. The size of a proton or neutron is approximately 1 femtometer (10^-15 meters). Mesons and baryons are also made up of quarks and have similar sizes to protons and neutrons. The Higgs-Boson, on the other hand, is a fundamental particle that gives mass to other particles. Its size is also incredibly small, approximately 10^-18 meters. Is there anything else you'd like to know?"

In [107]:
count_tokens(
    conversation_sum,
    "Yes it does.   Are there smaller particles than the ones you have already mentioned?"
)

Spent a total of 986 tokens


"Yes, there are smaller particles than the ones I mentioned. For example, there are leptons, which include electrons, muons, and tau particles. There are also neutrinos, which are extremely small and have no charge. Additionally, there are various types of bosons, which are particles that mediate fundamental forces like the electromagnetic force and the weak force. These include the photon, the W and Z bosons, and the gluon. However, it's important to note that our understanding of subatomic particles is constantly evolving, and there may be even smaller particles that we have not yet discovered or fully understood."

In [108]:
count_tokens(
    conversation_sum,
    "remind me what I am seeking help with?"
)

Spent a total of 584 tokens


'I apologize for getting carried away with my explanation of subatomic particles. Please let me know what you need help with and I will do my best to assist you.'

In [109]:
print(conversation_sum.memory.buffer)

The AI greets the human and explains the structure of subatomic particles. The human asks about their size and relation to other particles. The AI explains that our understanding of subatomic particles is constantly evolving and there may be even smaller particles we haven't discovered. The human asks for help, and the AI apologizes for getting carried away and offers to assist with their request.


Counting conversation length

In [110]:
# initialize tokenizer
tokenizer = tiktoken.encoding_for_model('gpt-3.5-turbo')

# show number of tokens for the memory used by each memory type
print(
    f'Buffer memory conversation length: {len(tokenizer.encode(conversation_buf.memory.buffer))}\n'
    f'Summary memory conversation length: {len(tokenizer.encode(conversation_sum.memory.buffer))}'
)

Buffer memory conversation length: 487
Summary memory conversation length: 75
